Dies sind die Hashtags, nach denen ich die Twitter API gefragt habe. Sie setzen sich zusammen aus den Hashtags in Tweets, die ich von Maassens Timeline am 20.08. abfragen konnte. Es handelt sich um die Hashtags, die häufiger als 2 mal und nicht häufiger als 10 mal auftreten.

In [ ]:
['5g',
 'akk',
 'asyl',
 'berufspolitiker',
 'bosbach',
 'bundespolizei',
 'burkinis',
 'carolarackete',
 'cdu',
 'change',
 'framing',
 'grundgesetz',
 'habeck',
 'integration',
 'ka',
 'kühnert',
 'laschet',
 'linke',
 'maaßen',
 'matthiasrößler',
 'mauerbau',
 'mauermörder',
 'merkel',
 'nahles',
 'partieausschlussverfahren',
 'ramelow',
 'sachsen',
 'sicherheit',
 'stasi',
 'strobl',
 'tauber',
 'tm',
 'union',
 'werder',
 'werteunion',
 'wertunion',
 'wirfürmaassen',
 'wirfürmaaßen',
 'wirsinddiecducsu',
 'zdf',
 'zweierleimass']

## Pakete und Daten

In [1]:
import data_handler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplot
mplot.rcParams['figure.figsize'] = [20, 15]
mplot.rcParams.update({'font.size': 22})

In [2]:
james = data_handler.ProjectManager('germ_tweets')
james.load_tweets(project_name='germ_tweets')

In [17]:
tweets = james.projects['germ_tweets']['raw_tweets_json']
data = james.transformer.order_by_date(tweets).drop_duplicates(['tweet_id', 'uid'])
data = data.set_index('datetime')[:'2019-08-15-07-00-00'].reset_index()

In [19]:
data.datetime.min()

Timestamp('2019-08-11 00:57:31')

In [20]:
data.datetime.max()

Timestamp('2019-08-15 07:59:59')

In [45]:
print(len(data))
print(len(list(set(data.uid))))

145892
44965


```data_handler``` ist ein Tool, dass ich für mich für die Arbeit mit Twitterdaten gebaut habe. Ende August werde ich es wohl veröffentlichen (zurzeit noch schlimmer Spaghetticode). Neben der Funktion als simpler Wrapper um ```twarc``` verarbeitet ```data_handler``` Tweet-JSON in (für meine Arbeit) nützliche ```pandas```-Dataframes.

Hier ein Auszug mit Tweets von Martin Sonneborn als Beispiel:

In [7]:
data[ data.screenname == 'MartinSonneborn' ].head()

,favorite_count,follower_count,friend_count,full_text,hashtags,is_quote,is_reply,is_retweet,mentions,quote_of_screenname,...,retweeted_user_creation_date,retweeted_user_status_count,screenname,status_count,text,tweet_id,uid,urls,user_creation_date,verified
datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-12 08:00:40,0,255426,2,"RT @DiePARTEI: ""Kein deutscher Ministerpräside...",[],False,False,True,[DiePARTEI],None,...,Fri Aug 14 10:25:43 +0000 2009,3341,MartinSonneborn,8278,None,1160823400711041024,20463983,[],Mon Feb 09 20:28:18 +0000 2009,True
2019-08-14 14:12:20,0,255426,2,RT @DiePARTEI_KMS: Wieder wählbar! Diesmal wir...,"[sltw19, sachsen, uweforfuture, DiePARTEI, nie...",False,False,True,[DiePARTEI_KMS],None,...,Sun Feb 26 12:34:46 +0000 2017,845,MartinSonneborn,8278,None,1161641710281207808,20463983,[],Mon Feb 09 20:28:18 +0000 2009,True


In [21]:
print(len(data[ data.screenname == 'HGMaassen' ]))
print(len(data[ data.retweet_of_screenname == 'HGMaassen' ]))

8
788


## Weitere Aufbereitung

Liste von HGMaassen-Retweetern erstellen:

In [24]:
users = list(set(data[ data.retweet_of_screenname == 'HGMaassen'].screenname))
print(len(users))

569


Folgender Code errechnet für die beiden Gruppen hg_retweeters (HGMaassen-Retweeter) und nohg_retweeters (Nicht-HGMaassen-Retweeter), mit welcher Wahrscheinlichkeit andere Screennames retweetet werden.

Dazu wird binär erfasst, ob ein User in der jeweiligen Gruppe andere User retweetet. Die von diesem retweeteten bekommen dann eine 1, alle anderen (für diesen User) eine 0. Das wird dann aufsummiert pro retweetetem Screenname. Schlußendlich wird es durch die Anzahl der User in der jeweiligen Gruppe geteilt, was die Proportion ergibt. Etwas umständlicher Code, aber erfüllt seinen Zweck.

In [25]:
len_all = len(list(set(data[ data.is_retweet].screenname)))
len_us = len(users)
len_nohg = len_all - len_us

hg_retweeters = data[ data.screenname.isin(users) & data.is_retweet].groupby(['screenname', 'retweet_of_screenname']).agg({'tweet_id':'count'}).unstack(fill_value=0).stack().reset_index()
hg_retweeters.loc[ hg_retweeters.tweet_id != 0 , 'tweet_id'] = 1
hg_retweeters = hg_retweeters.groupby('retweet_of_screenname').agg('sum').fillna(0).reset_index()
hg_retweeters['tweet_id'] = hg_retweeters.tweet_id.apply(lambda x: x / len_us).sort_values(ascending=True)
hg_retweeters = hg_retweeters.rename(columns={'tweet_id':'hg_rt'})

nohg_retweeters = data[ -data.screenname.isin(users) & data.is_retweet].groupby(['screenname', 'retweet_of_screenname']).agg({'tweet_id':'count'}).unstack(fill_value=0).stack().reset_index()
nohg_retweeters.loc[ nohg_retweeters.tweet_id != 0 , 'tweet_id'] = 1
nohg_retweeters = nohg_retweeters.groupby('retweet_of_screenname').agg('sum').fillna(0).reset_index()
nohg_retweeters['tweet_id'] = nohg_retweeters.tweet_id.apply(lambda x: x / len_nohg).sort_values(ascending=True)
nohg_retweeters = nohg_retweeters.rename(columns={'tweet_id':'nohg_rt'})

all_retweeters = data[ data.is_retweet].groupby(['screenname', 'retweet_of_screenname']).agg({'tweet_id':'count'}).unstack(fill_value=0).stack().reset_index()
all_retweeters.loc[ all_retweeters.tweet_id != 0 , 'tweet_id'] = 1
all_retweeters = all_retweeters.groupby('retweet_of_screenname').agg('sum').fillna(0)
all_retweeters['tweet_id'] = all_retweeters.tweet_id.apply(lambda x: x / len_all).sort_values(ascending=True)
all_retweeters = all_retweeters.rename(columns={'tweet_id':'all_rt'})


In [27]:
retweeters = nohg_retweeters.join(hg_retweeters.set_index('retweet_of_screenname'), on='retweet_of_screenname').join(all_retweeters, on='retweet_of_screenname').fillna(0)

In [51]:
retweeters[ (retweeters.nohg_rt > 0.01) | (retweeters.hg_rt > 0.01) ].sort_values('nohg_rt', ascending=False).fillna(0).head(20).reset_index(drop=True).loc[1:,].round(3)

,retweet_of_screenname,nohg_rt,hg_rt,all_rt
1,kaffeecup,0.057,0.009,0.056
2,SPIEGELONLINE,0.053,0.018,0.052
3,extra3,0.038,0.005,0.037
4,Steinhoefel,0.032,0.385,0.039
5,VQuaschning,0.030,0.005,0.029
6,reitschuster,0.027,0.327,0.033
7,sixtus,0.025,0.002,0.025
8,RolandTichy,0.024,0.355,0.031
9,FridayForFuture,0.022,0.000,0.022
10,heuteshow,0.022,0.002,0.021


Ich habe alle fehlenden Werte (gab es nur für die HGM-Retweeter) mit 0 aufgefüllt. Prüfen, ob NA tatsächlich mit "keine Retweets" gleichzusetzen ist.

In [37]:
print('Wurfschuh' in (data[ data.screenname.isin(users) & data.is_retweet ].retweet_of_screenname))
print('kaffeecup' in (data[ data.screenname.isin(users) & data.is_retweet ].retweet_of_screenname))
print('FridaysForFuture' in (data[ data.screenname.isin(users) & data.is_retweet ].retweet_of_screenname))

False
False
False


## Tabellen

In [42]:
retweeters.loc[ (retweeters.nohg_rt > 0.01) | (retweeters.hg_rt > 0.01) , ['retweet_of_screenname', 'hg_rt', 'nohg_rt', 'all_rt']].sort_values('hg_rt', ascending=False).head(20).round(3)

,retweet_of_screenname,hg_rt,nohg_rt,all_rt
3437,Steinhoefel,0.385,0.032,0.039
3090,RolandTichy,0.355,0.024,0.031
6041,reitschuster,0.327,0.027,0.033
5431,lawyerberlin,0.285,0.021,0.027
3609,TichysEinblick,0.220,0.011,0.015
2319,MalteKaufmann,0.213,0.013,0.017
5208,janfleischhauer,0.209,0.020,0.024
3311,ShakRiet,0.193,0.014,0.017
1056,Eddie_1412,0.183,0.009,0.013
147,AfD,0.178,0.015,0.018


In [44]:
retweeters[ (retweeters.nohg_rt > 0.01) | (retweeters.hg_rt > 0.01) ].sort_values('nohg_rt', ascending=False).head(20).round(3).to_html('nohg_rts.html', index=False)
retweeters.loc[ (retweeters.nohg_rt > 0.01) | (retweeters.hg_rt > 0.01) , ['retweet_of_screenname', 'hg_rt', 'nohg_rt', 'all_rt']].sort_values('hg_rt', ascending=False).head(20).round(3).to_html('hg_rts.html', index=False)

## Wie ungewöhnlich sind die Werte?

In [65]:
print(retweeters.nohg_rt.describe(percentiles=[.8,.9,.95,.98,.99, .995, .999, .9998]))
print(retweeters.hg_rt.describe(percentiles=[.8,.9,.95,.98,.99, .995, .999, .9998]))
print(retweeters.all_rt.describe(percentiles=[.8,.9,.95,.98,.99]))

count     6735.000000
mean         0.000370
std          0.002159
min          0.000037
50%          0.000075
80%          0.000224
90%          0.000559
95%          0.001193
98%          0.002956
99%          0.005292
99.5%        0.008794
99.9%        0.025787
99.98%       0.055429
max          0.110010
Name: nohg_rt, dtype: float64
count     6735.000000
mean         0.002401
std          0.014279
min          0.000000
50%          0.000000
80%          0.000000
90%          0.003515
95%          0.007030
98%          0.026924
99%          0.055641
99.5%        0.098998
99.9%        0.197529
99.98%       0.345257
max          0.384886
Name: hg_rt, dtype: float64
count    6735.000000
mean        0.000412
std         0.002263
min         0.000036
50%         0.000073
80%         0.000255
90%         0.000620
95%         0.001361
98%         0.003503
99%         0.006300
max         0.107762
Name: all_rt, dtype: float64
